# TimeGPT

> Unlock the power of accurate predictions and confidently navigate uncertainty. Reduce uncertainty and resource limitations. With TimeGPT, you can effortlessly access state-of-the-art models to make data-driven decisions. Whether you're a bank forecasting market trends or a startup predicting product demand, TimeGPT democratizes access to cutting-edge predictive insights, eliminating the need for a dedicated team of machine learning engineers.

In [ ]:
#| default_exp timegpt

In [ ]:
#| hide 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import json
import requests
from typing import Dict, List, Optional

import pandas as pd

In [ ]:
#| hide
import os

from dotenv import load_dotenv
from fastcore.test import test_eq, test_fail
from nbdev.showdoc import show_doc

load_dotenv()

True

In [ ]:
#| export
class TimeGPT:
    """
    A class used to interact with the TimeGPT API.
    """

    def __init__(self, token: str, api_url: str):
        """
        Constructs all the necessary attributes for the TimeGPT object.

        Parameters
        ----------
        token : str
            The authorization token to interact with the TimeGPT API.
        api_url : str
            The base URL for the TimeGPT API.
        """
        self.token = token
        self.api_url = api_url

    @property
    def request_headers(self):
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "authorization": f"Bearer {self.token}"
        }
        return headers
        
    def _parse_response(self, response: str) -> Dict:
        """Parses responde."""
        try:
            resp = json.loads(response)
        except Exception as e:
            raise Exception(response)
        return resp

    def _input_size(self, freq: str):
        response_input_size = requests.post(
            f'{self.api_url}/timegpt_input_size',
            json={'freq': freq}, 
            headers=self.request_headers,
        )
        response_input_size = self._parse_response(response_input_size.text)
        return response_input_size['data']

    def _preprocess_inputs(
            self, 
            df: pd.DataFrame, 
            X_df: Optional[pd.DataFrame] = None,
        ):
        y_cols = ['unique_id', 'ds', 'y']
        y = df[y_cols].to_dict(orient='split', index=False)
        x_cols = df.drop(columns='y').columns
        if dict(x_cols) - dict(y_cols):
            x = pd.concat([df[x_cols], X_df]).sort_values(['unique_id', 'ds'])
            x = x.to_dict(orient='split', index=False)
        else:
            x = None
        return y, x

    def forecast(
            self,
            df: pd.DataFrame,
            h: int,
            freq: str,
            X_df: Optional[pd.DataFrame] = None,
            level: Optional[List[int]] = None,
            finetune_steps: int = 0,
            clean_ex_first: bool = True,
        ):
        """Forecast your time series using TimeGPT.

        Parameters
        ----------
        df : pandas.DataFrame
            DataFrame with columns [`unique_id`, `ds`, `y`] and exogenous.
            If None, the `StatsForecast` class should have been instantiated
            using `df`.
        h : int
            Forecast horizon.
        freq : str
            Frequency of the data.
            See [pandas' available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
        X_df : pandas.DataFrame, optional (default=None)
            DataFrame with [`unique_id`, `ds`] columns and `df`'s future exogenous.
        level : List[float], optional (default=None)
            Confidence levels between 0 and 100 for prediction intervals.
        finetune_steps : int (default=0)
            Number of steps used to finetune TimeGPT in the
            new data.
        clean_ex_first : bool (default=True)
            Clean exogenous signal before making forecasts
            using TimeGPT.
        
        Returns
        -------
        fcsts_df : pandas.DataFrame
            DataFrame with TimeGPT forecasts for point predictions and probabilistic
            predictions (if level is not None).
        """
        pass

In [ ]:
show_doc(TimeGPT.__init__, title_level=1, name='TimeGPT')

---

# TimeGPT

>      TimeGPT (token:str, api_url:str)

Constructs all the necessary attributes for the TimeGPT object.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| token | str | The authorization token to interact with the TimeGPT API. |
| api_url | str | The base URL for the TimeGPT API. |

You can instantiate the `TimeGPT` class providing your credentials.

In [ ]:
timegpt = TimeGPT(
    token=os.environ['TIMEGPT_TOKEN'],
    api_url=os.environ['TIMEGPT_API_URL'],
)

In [ ]:
#| hide
# test input_size
test_eq(
    timegpt._input_size('D'),
    28,
)

In [ ]:
show_doc(TimeGPT.forecast, title_level=2)

---

## TimeGPT.forecast

>      TimeGPT.forecast (h:int, df:pandas.core.frame.DataFrame,
>                        X_df:Optional[pandas.core.frame.DataFrame]=None,
>                        level:Optional[List[int]]=None, finetune_steps:int=0,
>                        clean_ex_first:bool=True)

Forecast your time series using TimeGPT.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| h | int |  | Forecast horizon. |
| df | DataFrame |  | DataFrame with columns [`unique_id`, `ds`, `y`] and exogenous.<br>If None, the `StatsForecast` class should have been instantiated<br>using `df`. |
| X_df | Optional | None | DataFrame with [`unique_id`, `ds`] columns and `df`'s future exogenous. |
| level | Optional | None | Confidence levels between 0 and 100 for prediction intervals. |
| finetune_steps | int | 0 | Number of steps used to finetune TimeGPT in the<br>new data. |
| clean_ex_first | bool | True | Clean exogenous signal before making forecasts<br>using TimeGPT. |
| **Returns** | **pandas.DataFrame** |  | **DataFrame with TimeGPT forecasts for point predictions and probabilistic<br>predictions (if level is not None).** |